# Дообучение RuBert для определения спам сообщений учитывая числовые данные

## Импорт необходимых библиотек

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report

In [2]:
import sys
sys.path.append('../utils')
from preprocessing import preprocess_text, count_emojis, count_whitespaces, count_links, count_tags

In [3]:
import json

## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


## Обучение модели

### Подготовка выборок

In [6]:
text_data = df["text_preprocessed"].tolist()

In [7]:
numeric_data = df[["emojis", "whitespaces", "links", "tags"]].values

In [8]:
labels = df["label"].values

In [9]:
# 2. Токенизация текста
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
tokenized = tokenizer(
    text_data,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [10]:
numeric_features = torch.tensor(numeric_data, dtype=torch.float32)

In [11]:
labels = torch.tensor(labels, dtype=torch.long)

Определение датасета

In [12]:
class TextNumericDataset(Dataset):
    def __init__(self, tokenized_text, numeric_features, labels):
        self.input_ids = tokenized_text["input_ids"]
        self.attention_mask = tokenized_text["attention_mask"]
        self.numeric_features = numeric_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "numeric_features": self.numeric_features[idx],
            "label": self.labels[idx],
        }

In [13]:
dataset = TextNumericDataset(tokenized, numeric_features, labels)

In [14]:
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

### Определение модели

In [15]:
class RuBERTWithNumeric(nn.Module):
    def __init__(self, text_model_name, numeric_input_dim, hidden_dim=256, output_dim=2):
        super(RuBERTWithNumeric, self).__init__()
        self.bert = AutoModel.from_pretrained(text_model_name)
        self.bert_hidden_size = self.bert.config.hidden_size
        self.numeric_fc = nn.Sequential(
            nn.Linear(numeric_input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, hidden_dim)
        )
        self.classifier = nn.Linear(self.bert_hidden_size + hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask, numeric_features):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = bert_output.last_hidden_state[:, 0, :]
        numeric_embedding = self.numeric_fc(numeric_features)
        combined_features = torch.cat((cls_embedding, numeric_embedding), dim=1)
        output = self.classifier(combined_features)
        return output

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
device

device(type='cuda')

Инициализация и обучение модели

In [18]:
model = RuBERTWithNumeric("cointegrated/rubert-tiny2", numeric_input_dim=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        numeric_features = batch["numeric_features"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, numeric_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Эпоха {epoch + 1} завершена, Средние потери: {total_loss / len(train_loader):.4f}")

Эпоха 1 завершена, Средние потери: 0.0202
Эпоха 2 завершена, Средние потери: 0.0034
Эпоха 3 завершена, Средние потери: 0.0011
Эпоха 4 завершена, Средние потери: 0.0005
Эпоха 5 завершена, Средние потери: 0.0004


## Сохранение

In [20]:
torch.save(model.state_dict(), "../models/finetuned_rubert_tiny2_with_numeric/model.pth")
print("Модель сохранена в ../models/finetuned_rubert_tiny2_with_numeric/model.pth")

Модель сохранена в ../models/finetuned_rubert_tiny2_with_numeric/model.pth


In [21]:
model_config = {
    "text_model_name": "cointegrated/rubert-tiny2",
    "numeric_input_dim": 4,
    "hidden_dim": 256,
    "output_dim": 2
}

In [22]:
with open("../models/finetuned_rubert_tiny2_with_numeric/config.json", "w") as f:
    json.dump(model_config, f)
print("Конфигурация модели сохранена в ../models/finetuned_rubert_tiny2_with_numeric/config.json")

Конфигурация модели сохранена в ../models/finetuned_rubert_tiny2_with_numeric/config.json


In [23]:
tokenizer.save_pretrained('../models/finetuned_rubert_tiny2_with_numeric/')
print("Токенизатор сохранён в директорию ../models/finetuned_rubert_tiny2_with_numeric/")

Токенизатор сохранён в директорию ../models/finetuned_rubert_tiny2_with_numeric/


## Загрузка

Тест загрузки модели и её архитектуры

In [24]:
with open("../models/finetuned_rubert_tiny2_with_numeric/config.json", "r") as f:
    loaded_config = json.load(f)

In [25]:
loaded_model = RuBERTWithNumeric(
    text_model_name=loaded_config["text_model_name"],
    numeric_input_dim=loaded_config["numeric_input_dim"],
    hidden_dim=loaded_config["hidden_dim"],
    output_dim=loaded_config["output_dim"]
).to(device)
loaded_model.load_state_dict(torch.load("../models/finetuned_rubert_tiny2_with_numeric/model.pth"))
loaded_model.eval()
print("Модель успешно загружена")

Модель успешно загружена


C:\Users\overklassniy\AppData\Local\Temp\ipykernel_3728\3999636723.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("../models/fin

In [26]:
loaded_tokenizer = AutoTokenizer.from_pretrained("../models/finetuned_rubert_tiny2_with_numeric/")
print("Токенизатор успешно загружен из директории ../models/finetuned_rubert_tiny2_with_numeric/")

Токенизатор успешно загружен из директории ../models/finetuned_rubert_tiny2_with_numeric/


##### Без предобработки данных

In [ ]:
rb_trainer.train()

In [ ]:
rb_trainer.evaluate()

Сохранение модели

In [ ]:
rb_model.save_pretrained("../models/finetuned_rubert_tiny2")
tokenizer.save_pretrained("../models/finetuned_rubert_tiny2")

##### С предобработкой данных

In [ ]:
rb_trainer_p.train()

In [ ]:
rb_trainer_p.evaluate()

Сохранение модели

In [ ]:
rb_model.save_pretrained("../models/finetuned_rubert_tiny2_p")
tokenizer.save_pretrained("../models/finetuned_rubert_tiny2_p")

## Оценка качества модели

In [28]:
# Тестовые сообщения
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!"
]

In [29]:
tokenized = loaded_tokenizer(
    test_messages,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [30]:
numeric_data = [[count_emojis(d), count_whitespaces(d), count_links(d), count_tags(d)] for d in test_messages]

In [31]:
numeric_features = torch.tensor(numeric_data, dtype=torch.float32).to(device)

In [39]:
# Передача данных в модель
with torch.no_grad():
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    outputs = loaded_model(input_ids, attention_mask, numeric_features)
    predictions = torch.argmax(outputs, dim=1)
    probabilities = F.softmax(outputs, dim=1)

In [40]:
results_pred = predictions.cpu().numpy()
results_proba = probabilities.cpu().numpy()

In [45]:
for message_i in range(len(test_messages)):
    pred = results_pred[message_i]
    probas = results_proba[message_i]
    print(f"Сообщение: {test_messages[message_i]}")
    print(f"Класс: {pred}")
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probas]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9999695, 3.06e-05]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0006408, 0.9993592]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 1
Вероятности: [0.0091884, 0.9908117]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.980187, 0.019813]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9999599, 4.01e-05]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.00030